In [3]:
import networkx as nx
import numpy as np
import math

class NMI:
    def __init__(self, graph, cover1, cover2):
        self.graph=graph
        self.cover1=cover1
        self.cover2=cover2
    
    def h_w_n (self,w,n):
        try:
            return -w*math.log(w/n,2)
        except:
            return 0

    def create_clust_matrix(self,cluster_list,graph):

        nodes=list(graph.nodes())
        n_nodes=len(nodes)
        mat=np.zeros((n_nodes,len(clusters)))

        for ind,node in enumerate(nodes):
            for clust_ind,clust in enumerate(cluster_list):
                if node in clust:
                    mat[ind,clust_ind]=1
                else:
                    continue
        return mat
    def calculate_H_X_given_Y (self,graph, cover_1, cover_2):
    
        X_i=self.create_clust_matrix(cover_1,graph)
        Y_j=self.create_clust_matrix(cover_2,graph)
        nodes=list(graph.nodes())
        n_nodes=len(nodes)
        abcd=np.zeros((4,len(cover_1)*len(cover_2))) #cols are combinations of the clusters i,j and rows are a,b,c,d
        index_col_abcd=0
        H_i_j_XY=np.zeros(len(cover_1)*len(cover_2))

        for i in range(len(cover_1)):
            for j in range(len(cover_2)):
                clust_i=X_i[:,i]
                clust_j=Y_j[:,j]

                abcd_vect=np.zeros(4)

                for m in range(n_nodes):
                    if clust_i[m]==0 and clust_j[m]==0:
                        abcd_vect[0] += 1
                    elif clust_i[m]==0 and clust_j[m]==1:
                        abcd_vect[1] += 1
                    elif clust_i[m]==1 and clust_j[m]==0:
                        abcd_vect[2] += 1
                    elif clust_i[m]==1 and clust_j[m]==1:
                        abcd_vect[3] += 1
                H_i_j_XY[index_col_abcd]=h_w_n(abcd_vect[0],n_nodes)+h_w_n(abcd_vect[1],n_nodes)+h_w_n(abcd_vect[2],n_nodes)+h_w_n(abcd_vect[3],n_nodes)-h_w_n(abcd_vect[1]+abcd_vect[3],n_nodes)-h_w_n(abcd_vect[0]+abcd_vect[2],n_nodes)
                abcd[:,index_col_abcd]=abcd_vect
                index_col_abcd +=1

        H_i=np.zeros(len(cover_1))
        count=0

        for j in range(len(cover_1)):
            H_i[j]=min(H_i_j_XY[count:count+len(cover_2)])
        H_XY=np.sum(H_i) 

        return H_XY

    def calculate_H_Y_given_X(self,graph, cover_1, cover_2):
    
        X_i=self.create_clust_matrix(cover_1,graph)
        Y_j=self.create_clust_matrix(cover_2,graph)
        nodes=list(graph.nodes())
        n_nodes=len(nodes)
        abcd=np.zeros((4,len(cover_1)*len(cover_2))) #cols are combinations of the clusters i,j and rows are a,b,c,d
        index_col_abcd=0
        H_i_j_YX=np.zeros(len(cover_1)*len(cover_2))

        for i in range(len(cover_1)):
            for j in range(len(cover_2)):
                clust_i=X_i[:,i]
                clust_j=Y_j[:,j]

                abcd_vect=np.zeros(4)

                for m in range(n_nodes):
                    if clust_j[m]==0 and clust_i[m]==0:
                        abcd_vect[0] += 1
                    elif clust_j[m]==0 and clust_i[m]==1:
                        abcd_vect[1] += 1
                    elif clust_j[m]==1 and clust_i[m]==0:
                        abcd_vect[2] += 1
                    elif clust_j[m]==1 and clust_i[m]==1:
                        abcd_vect[3] += 1
                H_i_j_YX[index_col_abcd]=h_w_n(abcd_vect[0],n_nodes)+h_w_n(abcd_vect[1],n_nodes)+h_w_n(abcd_vect[2],n_nodes)+h_w_n(abcd_vect[3],n_nodes)-h_w_n(abcd_vect[1]+abcd_vect[3],n_nodes)-h_w_n(abcd_vect[0]+abcd_vect[2],n_nodes)
                abcd[:,index_col_abcd]=abcd_vect
                index_col_abcd +=1
        H_j=np.zeros(len(cover_2))
        count=0
        for i in range(len(cover_2)):
            H_j[i]=min(H_i_j_YX[count:count+len(cover_1)])
        H_YX=np.sum(H_j) 
        return H_YX
    def calculate_H_cover (self,graph,cover):
    
        H_Y=0
        n_nodes=len(list(graph.nodes()))

        for i in range(len(cover)):
            H_Y += h_w_n(len(cover[i]),n_nodes) + h_w_n(n_nodes-len(cover[i]),n_nodes)

        return H_Y
    def calculate_NMI (self):
    
        H_XY=self.calculate_H_X_given_Y(self.graph,self.cover1,self.cover2)
        H_YX=self.calculate_H_Y_given_X(self.graph,self.cover1,self.cover2)
        H_X=self.calculate_H_cover (self.graph,self.cover1)
        H_Y=self.calculate_H_cover (self.graph,self.cover2)
        NMI=1-0.5*((H_XY/H_X)+(H_YX/H_Y))

        return NMI